**Student ID: u6579712** 

**Student Name: Jieli Zheng**

**Task-3: Face Recognition using Eigenface**
--

In [1]:
import os
import cv2
import numpy as np
import math
from matplotlib import pyplot as plt
from matplotlib.pyplot import imread

In [2]:
# calculate the mean face and normalized faces
def preprocess(A_pp):
    A = None
    Q_norms = None
    A_means = None


    A_pp_mean = np.mean(A_pp, axis=1, keepdims=True)
    Q = A_pp - A_pp_mean
    Q_norms = np.amax(np.abs(Q), axis=1, keepdims=True)
    A = Q / np.where(Q_norms == 0, 1, Q_norms)
    A_means = np.mean(A_pp, axis=1, keepdims=True)

    return A, Q_norms, A_means


# the main algorithm of eigenface.
def eigen_faces(A_pp):
    C = None
    F = None
    D = None
    Q_norms = None
    A_means = None


    A, Q_norms, A_means = preprocess(A_pp)
    D, C = np.linalg.eig(A.T @ A)
    D = D.real
    C = C.real


    F = A @ C
    F_norm = np.linalg.norm(F, axis=0, keepdims=True)
    F = F / F_norm

    C = A.T @ F
    return A, C, F, D, Q_norms, A_means





In [3]:
def reconstruct_image(Img, F, Q_norms, A_means,A):
    R = np.zeros((231, 195))


    #     calculate C, which is AT@A's eigenvector matrix
    C = A.T @ F

    #     calculate R_vector
    R_vector = np.zeros(Q_norms.shape)

    for j in range(len(Img)):
        R_vector = R_vector + (Img[j] * Q_norms * F[:, j].reshape(-1, 1))

    R_vector += A_means
    R = R_vector.reshape((231, 195), order="F")

    return R



def reduce_dimensionality(image_vector, k, F, D, A_means, Q_norms):
    compressed_image = None
    p = None


    #     k:k paras int
    #     F:eigen Face， A@A.T   eigenvector 45045,135
    #     D: A.T@A eigenvalue 135,
    #     A_means: average face lookings 45045,1
    #     Q_norms: image normalized       45045,1
    #     image_vector: pic we need to compress ,45045,1

    #     intialize image_vector

    Q = image_vector.reshape(-1, 1) - A_means
    image_normalized = Q / np.where(Q_norms == 0, 1, Q_norms)

    #     handle compressed image
    F_k = F[:, 0:k]

    compressed_parameter = F_k.T @ image_normalized
    compressed_parameter_s = np.zeros((135, 1))
    for i in range(135):
        if (i < k):
            compressed_parameter_s[i] = compressed_parameter[i]


    #     handle p
    d_sum = np.sum(D)
    fenzi = 0
    for i in range(k):
        fenzi += D[i]
    p = fenzi / d_sum

    print(image_normalized.shape)
    print(compressed_parameter.shape)
    print(F_k.shape)


    return compressed_parameter_s, p